In [60]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


In [61]:
#imports 
import os
import nltk
import spacy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.onnx as onnx
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as T
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from torch.utils.data import DataLoader,Dataset
from PIL import Image
from tqdm import tqdm
from time import sleep

from src.baseline.vocabulary import Vocabulary
from src.baseline.data_loader import get_loader, get_mean, get_std
from src.baseline.coa_model import get_new_model
from src.armoria_api_generator_helper import ArmoriaAPIGeneratorHelper

from pyinstrument import Profiler
from datetime import datetime
from shutil import copyfile


In [62]:
#Hyperparams
embed_size=300
vocab_size = 36
attention_dim=256
encoder_dim=2048
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device)


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 11.91 GiB total capacity; 5.90 MiB already allocated; 4.25 MiB free; 6.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Generate caption file from Real dataset in CROPPED folder

In [69]:
REAL_DATASET='/home/salnabulsi/coat-of-arms/data/cropped_coas/out_valid'
test_annotation_file  = REAL_DATASET + '/test_real_captions.txt'
test_annotation_psumsq_file  = REAL_DATASET + '/test_real_captions_psumsq.txt'
data_location = REAL_DATASET + '/images'

In [70]:
from os import listdir
from os.path import isfile, join
images_names = [f.split('.jpg')[0] for f in listdir(data_location) if isfile(join(data_location, f))]
images_names

['B AG tierced per fess',
 'A G 2 piles each tipped with a bluebell',
 ' semy of fleurs-de-lis_ 3 lions passt guard_ =_ =  {BO, GO}',
 'S A qtly',
 'O GV elm branch with 3 leaves erect',
 'B O 3 fleurs-de-lis',
 'A OG lion holding cross formy',
 'O VO bendy semy of mullets',
 'B O per chevron flory at top',
 'B AOG book betw 3 fleurs-de-lis and over all a bend',
 'G A cross potenty',
 ' 3 fleurs-de-lis_ 3 lions passt guard_ =_ = _ book  {BO, GO_ A}',
 'O G lion rampant',
 'B O semy of fleurs-de-lis',
 'O G lozengy',
 'O S talbot_s head',
 'G ABN 2 keys in saltire _ chief ch. book',
 ' stag passt, 2 barbels affronted_ = _ =   {OS, AG}',
 'A S fess',
 'B OO cross acc. 4 fire steels',
 'O X AB per chevron plain _ lozengy',
 'O G bend',
 'A S 2 bars',
 'B O rose',
 'B AO cross and crozier sn',
 ' barry, plain   {AG, B}',
 'O S talbot gorged',
 'S EE cross engrailed acc. crescent in chf dx',
 ' cross of Jerusalem_ barry _ lion_ lion_ =   {AO, XGAB, OG}',
 'A S pouch',
 'G AV elephant on ter

In [71]:
api_gen_helper = ArmoriaAPIGeneratorHelper(caption_file=test_annotation_file,
                                           folder_name=data_location,
                                           permutations=images_names)

In [72]:
api_gen_helper.creat_caption_file(test_annotation_file)

In [73]:
api_gen_helper.generate_cropped_caption_file()

In [74]:
api_gen_helper.add_pixels_column(REAL_DATASET, test_annotation_psumsq_file,test_annotation_file, start_index=1)

Invalid label: "semy of fleurs-de-lis_ 3 lions passt guard_ =_ =  {BO, GO}.jpg,semy of fleurs-de-lis_ 3 lions passt guard_ =_ =  {BO, GO}
" is skipped
Invalid label: "3 fleurs-de-lis_ 3 lions passt guard_ =_ = _ book  {BO, GO_ A}.jpg,3 fleurs-de-lis_ 3 lions passt guard_ =_ = _ book  {BO, GO_ A}
" is skipped
Invalid label: "stag passt, 2 barbels affronted_ = _ =   {OS, AG}.jpg,stag passt, 2 barbels affronted_ = _ =   {OS, AG}
" is skipped
Invalid label: "barry, plain   {AG, B}.jpg,barry, plain   {AG, B}
" is skipped
Invalid label: "cross of Jerusalem_ barry _ lion_ lion_ =   {AO, XGAB, OG}.jpg,cross of Jerusalem_ barry _ lion_ lion_ =   {AO, XGAB, OG}
" is skipped
Invalid label: "sun on mount_ 3 lion passt guard_ =_ =   {BOS, OS}.jpg,sun on mount_ 3 lion passt guard_ =_ =   {BOS, OS}
" is skipped
Invalid label: "barry_ lion q.f._ =_ =  {OS, GA}.jpg,barry_ lion q.f._ =_ =  {OS, GA}
" is skipped
Invalid label: "lozenge_ per pale_ =_ =    {GA, AG}.jpg,lozenge_ per pale_ =_ =    {GA, AG}
"

## Check which labels are valid according to COA automata